In [1]:
import pandas as pd
import math
import random
from collections import namedtuple
from PSSimPy.simulator import ABMSim

In [2]:
GameState = namedtuple("GameState", [
    "t",                       # current period
    "focal_balance",           # known exactly for the focal player
    "focal_borrowed",          # known exactly for the focal player
    "focal_obligations",       # known exactly for the focal player
    "estimated_others_bal",    # dict: player_id -> estimated balance
    "estimated_others_borrow", # dict: player_id -> estimated borrowed
    "estimated_others_oblig",  # dict: player_id -> estimated obligations
    "observed_data"           # dict or custom structure for partial obs about each other player
])

In [3]:
class OriGameTreeSearch:
    """
    Demonstration of a full traversal + backward induction from the focal player's
    perspective, with partial observation logic extended to all other players
    uniformly.
    """
    
    def __init__(self, n_players=3, focal_player=0):
        self.n_players = n_players
        self.focal_player = focal_player
        self.initial_state = self.initialize_state()
    
    def initialize_state(self):
        """Everyone starts at 0, partial observations start empty."""
        estimated_others_bal = {}
        estimated_others_borrow = {}
        estimated_others_oblig = {}
        observed_data = {}
        
        for p in range(self.n_players):
            if p != self.focal_player:
                estimated_others_bal[p] = 0
                estimated_others_borrow[p] = 0
                estimated_others_oblig[p] = 0
                observed_data[p] = {}  # e.g. could store period-by-period known transactions
                
        return GameState(
            t=0,
            focal_balance=0,
            focal_borrowed=0,
            focal_obligations=0,
            estimated_others_bal=estimated_others_bal,
            estimated_others_borrow=estimated_others_borrow,
            estimated_others_oblig=estimated_others_oblig,
            observed_data=observed_data
        )
    
    def all_action_profiles(self):
        """All possible pay/delay (0,1) combos for n players."""
        profiles = []
        for i in range(2**self.n_players):
            bits = []
            tmp = i
            for _ in range(self.n_players):
                bits.append(tmp % 2)
                tmp //= 2
            bits.reverse()
            profiles.append(bits)
        return profiles
    
    def is_terminal(self, state, n_periods):
        return state.t >= n_periods
    
    def apply_focal_actions(self, state, focal_action, delta, gamma):
        """
        Update the focal player's known state given the chosen action.
        Return (new_bal, new_borr, new_oblig, focal_immediate_cost).
        """
        bal = state.focal_balance
        borr = state.focal_borrowed
        obl = state.focal_obligations
        
        focal_immediate_cost = 0.0
        if focal_action == 1:  # pay
            if bal >= obl:
                bal -= obl
                obl = 0
            else:
                shortfall = obl - bal
                bal = 0
                borr += shortfall
                obl = 0
                focal_immediate_cost += gamma * shortfall
            # attempt repay
            if bal > 0 and borr > 0:
                repay = min(bal, borr)
                borr -= repay
                bal -= repay
        else:  # delay
            focal_immediate_cost += delta * obl
        
        return bal, borr, obl, focal_immediate_cost
    
    def estimate_other_players_next_state(self, state, joint_action, delta, gamma):
        """
        For each other player, incorporate partial observations from state.observed_data
        plus a heuristic for pay/delay. 
        """
        next_bal = dict(state.estimated_others_bal)
        next_bor = dict(state.estimated_others_borrow)
        next_obl = dict(state.estimated_others_oblig)
        
        for p in range(self.n_players):
            if p == self.focal_player:
                continue
            action = joint_action[p]
            
            # Incorporate partial observation from observed_data
            # e.g., if we saw that player p definitely paid X units last period, 
            # we update next_bal[p] accordingly. 
            # In a real scenario, you'd store that info in state.observed_data[p].
            
            # Then apply the heuristic:
            bal = next_bal[p]
            borr = next_bor[p]
            obl = next_obl[p]
            
            if action == 1:  # pay
                if bal >= obl:
                    bal -= obl
                    obl = 0
                else:
                    shortfall = obl - bal
                    bal = 0
                    borr += shortfall
                    obl = 0
                # repay if leftover
                if bal > 0 and borr > 0:
                    repay = min(bal, borr)
                    borr -= repay
                    bal -= repay
            else:  # delay
                # they'd add cost delta*obl, but we don't track others' costs in detail
                pass
            
            next_bal[p] = bal
            next_bor[p] = borr
            next_obl[p] = obl
        
        return next_bal, next_bor, next_obl
    
    def evolve_obligations(self, state, lambda_prob):
        """
        Add new obligations for each player with probability lambda_prob, 
        including the focal player. For partial observation, you could 
        store in observed_data if the focal player sees certain transactions.
        """
        # focal
        new_focal_obl = state.focal_obligations
        if random.random() < lambda_prob:
            new_focal_obl += 1
        
        # others
        new_others_obl = dict(state.estimated_others_oblig)
        new_observed_data = dict(state.observed_data)
        
        for p in range(self.n_players):
            if p == self.focal_player:
                continue
            if random.random() < lambda_prob:
                new_others_obl[p] += 1
                
                # Potentially record in new_observed_data[p] 
                # that p has new obligation arrivals, if such info is partially observable.
                # This snippet doesn't do that logic explicitly, but you could add it here.
        
        return new_focal_obl, new_others_obl, new_observed_data
    
    def game_tree_search(self, state, n_periods, delta, gamma, lambda_prob=0.5, memo=None):
        """
        Full traversal. We do a minimax approach from the focal player's perspective,
        enumerating all joint actions. Others' states are updated heuristically.
        """
        if memo is None:
            memo = {}
        
        # Make a key for memoization
        state_key = (
            state.t,
            state.focal_balance,
            state.focal_borrowed,
            state.focal_obligations,
            tuple(sorted(state.estimated_others_bal.items())),
            tuple(sorted(state.estimated_others_borrow.items())),
            tuple(sorted(state.estimated_others_oblig.items())),
            # Observed data can be large, so either skip or define a summary
            # For demonstration, we skip or do partial
        )
        
        if state_key in memo:
            return memo[state_key]
        
        # Terminal check
        if self.is_terminal(state, n_periods):
            memo[state_key] = (0.0, None)
            return 0.0, None
        
        # Enumerate all joint actions
        all_profiles = self.all_action_profiles()
        
        # We'll track best cost for each focal action under a "worst-case" view 
        cost_by_focal_action = {0: [], 1: []}
        
        for profile in all_profiles:
            focal_action = profile[self.focal_player]
            
            # 1) Focal update
            (new_focal_bal,
             new_focal_bor,
             new_focal_obl,
             focal_immediate_cost) = self.apply_focal_actions(state, focal_action, delta, gamma)
            
            # 2) Others update
            next_bal, next_bor, next_obl = self.estimate_other_players_next_state(state, profile, delta, gamma)
            
            # 3) Evolve obligations
            next_t = state.t + 1
            new_focal_obl2, new_others_obl2, new_obs_data2 = self.evolve_obligations(state, lambda_prob)
            
            # Overwrite with newly updated obligations
            new_focal_obl = new_focal_obl2
            for p in next_obl:
                next_obl[p] = new_others_obl2[p]
            
            # 4) Build child state
            child_state = GameState(
                t=next_t,
                focal_balance=new_focal_bal,
                focal_borrowed=new_focal_bor,
                focal_obligations=new_focal_obl,
                estimated_others_bal=next_bal,
                estimated_others_borrow=next_bor,
                estimated_others_oblig=next_obl,
                observed_data=new_obs_data2
            )
            
            # 5) Recurse
            future_cost, _ = self.game_tree_search(child_state, n_periods, delta, gamma, lambda_prob, memo)
            total_focal_cost = focal_immediate_cost + future_cost
            
            cost_by_focal_action[focal_action].append(total_focal_cost)
        
        # Summarize worst-case for each focal action:
        worst_case_delay = max(cost_by_focal_action[0]) if cost_by_focal_action[0] else float('inf')
        worst_case_pay   = max(cost_by_focal_action[1]) if cost_by_focal_action[1] else float('inf')
        
        if worst_case_delay < worst_case_pay:
            memo[state_key] = (worst_case_delay, 0)
        else:
            memo[state_key] = (worst_case_pay, 1)
        
        return memo[state_key]

In [ ]:

# Example usage
n_players = 5
n_periods = 5
delta = 1.0   # delay cost
gamma = 0.2   # borrowing fee
lambda_prob = 0.5

GameSearch = OriGameTreeSearch(n_players)

# We'll run a naive game tree search from the initial state
# (in a real setting, random events should be branched out, and 
# multi-player optimal strategy might need more advanced solution concepts)
initial_state = GameSearch.initial_state
best_cost, best_action = GameSearch.game_tree_search(initial_state, n_periods, delta, gamma, lambda_prob)

print(f"Focal player's worst-case total cost from t=0: {best_cost:.2f}")
print(f"Best immediate action at t=0: {'Delay' if best_action == 0 else 'Pay'}")

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1092f5400>>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/intradayliquiditygame/lib/python3.13/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
